In [88]:
import copy
from datetime import datetime
from time import sleep

from pytket import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction,MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends import Backend
from pytket.backends.backendresult import BackendResult
from pytket.extensions.qiskit import AerBackend

from pytket.utils.operators import QubitPauliOperator
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol

from qnexus.client import auth, circuits, projects, jobs
from qnexus.references import CircuitRef, ProjectRef



VQE example adapted from https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_variational_experiment_with_batching.ipynb

### TODO

- wait for job completion using websockets
- show how we can resume job after a crash by retrieving the correct circuits from the database

In [66]:
# 1. Synthesise Symbolic State-Preparation Circuit (hardware efficient ansatz)

symbols = [Symbol(f"p{i}") for i in range(4)]
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

#render_circuit_jupyter(symbolic_circuit)

[X q[0]; Ry(p1) q[1]; Ry(p0) q[0]; CX q[0], q[1]; Ry(p2) q[0]; Ry(p3) q[0]; ]

In [68]:
# 2. Define Hamiltonian 
# coefficients in the Hamiltonian are obtained from PhysRevX.6.031007

coeffs = [-0.4804, 0.3435, -0.4347, 0.5716, 0.0910, 0.0910]
term0 = {
    QubitPauliString(
        {
            Qubit(0): Pauli.I,
            Qubit(1): Pauli.I,
        }
    ): coeffs[0]
}
term1 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.I}): coeffs[1]}
term2 = {QubitPauliString({Qubit(0): Pauli.I, Qubit(1): Pauli.Z}): coeffs[2]}
term3 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): coeffs[3]}
term4 = {QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): coeffs[4]}
term5 = {QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): coeffs[5]}
term_sum = {}
term_sum.update(term0)
term_sum.update(term1)
term_sum.update(term2)
term_sum.update(term3)
term_sum.update(term4)
term_sum.update(term5)
hamiltonian = QubitPauliOperator(term_sum)



In [70]:
# 3.1 Computing Expectation Values for Pauli-Strings

def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)


# 3.2 Computing Expectation Values for sums of Pauli-strings multiplied by coefficients

def compute_expectation_value(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff > 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

In [90]:
class Objective:
    def __init__(
        self,
        nexus_project: ProjectRef,
        symbolic_circuit: CircuitRef,
        problem_hamiltonian: QubitPauliOperator,
        n_shots_per_circuit: int,
        iteration_number: int = 0,
        n_iterations: int = 10,
    ) -> None:
        r"""Returns the objective function needed for a variational
        procedure.
        """
        terms = [term for term in problem_hamiltonian._dict.keys()]
        self._backend=AerBackend()
        self._symbolic_circuit: Circuit = symbolic_circuit.get_circuit()
        self._hamiltonian: QubitPauliOperator = problem_hamiltonian
        self._nshots: int = n_shots_per_circuit
        self._measurement_setup: MeasurementSetup = measurement_reduction(
            terms, strat=PauliPartitionStrat.CommutingSets
        )
        self._nexus_project = nexus_project
        self._iteration_number: int = iteration_number
        self._niters: int = n_iterations

    def __call__(self, parameter: ndarray) -> float:
        value = self._objective_function(parameter)
        self._iteration_number += 1
        if self._iteration_number >= self._niters:
            self._iteration_number = 0
        return value
    
    def _objective_function(
        self,
        parameters: ndarray,
    ) -> float:
        assert len(parameters) == len(self._symbolic_circuit.free_symbols())
        circuit_list = self._build_circuits(parameters)

        # Execute circuits with Nexus
        execute_ref =jobs.submit_execute_job(
            name=f"execute_job_VQE_{datetime.now()}_{self._iteration_number}",
            circuits=circuit_list,
            project=self._nexus_project,
            n_shots=[self._nshots]*len(circuit_list)
        )
        # TODO actually wait for the job
        sleep(20)
        results = [item.get_result() for item in jobs.execution_results(execute_ref)]

        expval = compute_expectation_value(
            results, self._measurement_setup, self._hamiltonian
        )
        return expval
    
    def _build_circuits(self, parameters: ndarray) -> list[CircuitRef]:
        circuit = self._symbolic_circuit.copy()
        symbol_dict = {s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)}
        circuit.symbol_substitution(symbol_dict)

        properties = {str(sym): val for sym, val in symbol_dict.items()}
        properties["iteration"] = self._iteration_number

        # Upload the numerical state-prep circuit to Nexus
        circuits.submit(
            circuit=circuit, 
            project=self._nexus_project,
            name=f"state prep circuit {self._iteration_number}",
            description="A numerical state-prep circuit for my dihydrogen VQE",
            properties=properties
        )
        circuit_list = []
        for mc in self._measurement_setup.measurement_circs:
            c = circuit.copy()
            c.append(mc)
            # Upload each measurement circuit to Nexus with correct params
            measurement_circuit_ref = circuits.submit(
                circuit=c, 
                project=self._nexus_project,
                name=f"state prep circuit {self._iteration_number}",
                description="A numerical state-prep circuit for my dihydrogen VQE",
                properties=properties
            )
            circuit_list.append(measurement_circuit_ref)
        # compile circuits with Nexus
        compile_job_ref = jobs.submit_compile_job(
            name=f"compile_job_VQE_{datetime.now()}_{self._iteration_number}",
            circuits=circuit_list,
            optimisation_level=2,
            project=self._nexus_project,
        )
        # TODO actually wait for the job
        sleep(10)
        compiled_circuit_refs = jobs.compilation_results(compile_job_ref)
        return compiled_circuit_refs



In [91]:
# set up the project
project_ref = projects.submit(
    name=f"VQE_example_{str(datetime.now())}", 
    description="A VQE done with qexus", 
    properties={}
)

In [92]:
# Set up the properties
projects.add_property(
    project_ref, 
    name="iteration", 
    property_type="int", 
    description=f"The iteration number in my dihydrogen VQE experiment", 
    required=False
)

for sym in symbolic_circuit.free_symbols():
    sym_name = str(sym)
    projects.add_property(
        project_ref, 
        name=sym_name, 
        property_type="float", 
        description=f"Our VQE {sym_name} parameter", 
        required=False
    )

In [93]:
# Upload our ansatz circuit

ansatz_ref = circuits.submit(
    circuit=symbolic_circuit, 
    project=project_ref,
    name="ansatz_circuit",
    description="The ansatz state-prep circuit for my dihydrogen VQE",
    properties={}
)

In [94]:
objective = Objective(
    project_ref,
    ansatz_ref,
    hamiltonian,
    n_shots_per_circuit = 500,
    n_iterations= 2,
)

In [95]:
initial_parameters = random_sample(len(symbolic_circuit.free_symbols()))

result = minimize(
    objective,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)


   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    2   F =-6.824244E-01    MAXCV = 0.000000E+00
   X = 1.093379E+00   3.649019E-01   8.198273E-02   1.485567E-01


In [49]:
print(result.fun)
print(result.x)

-0.8685952
[1.88844242 1.30241341 0.93346525 0.01860803]
